# 06 — Data Wrangling
**Data Analysis Portfolio**

Topics: merge/join, concat, melt, pivot_table, groupby+transform, cut/qcut, string operations

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)
print('Ready.')

## 1. Setup — Multiple Tables

In [ ]:
employees = pd.DataFrame({
    'emp_id':    range(1001,1021),
    'name':      [f'Emp_{i}' for i in range(20)],
    'dept_id':   np.random.choice([10,20,30,40], 20),
    'salary':    np.random.randint(35000,120000,20),
    'hire_year': np.random.choice([2018,2019,2020,2021,2022,2023], 20),
})
departments = pd.DataFrame({
    'dept_id':  [10,20,30,40],
    'dept_name':['IT','HR','Finance','Marketing'],
    'location': ['Bangalore','Mumbai','Delhi','Chennai'],
    'budget':   [5000000,2000000,3500000,2800000],
})
performance = pd.DataFrame({
    'emp_id':   range(1001,1021),
    'q1_score': np.random.randint(50,100,20),
    'q2_score': np.random.randint(50,100,20),
    'q3_score': np.random.randint(50,100,20),
    'q4_score': np.random.randint(50,100,20),
})
print("employees:", employees.shape, "| departments:", departments.shape, "| performance:", performance.shape)

## 2. Merge / Join

In [ ]:
# inner join
inner = employees.merge(departments, on='dept_id', how='inner')
print("Inner join:", len(inner), "rows")
print(inner.head(3))

In [ ]:
# chain merge — 3 tables
full = employees.merge(departments, on='dept_id', how='left').merge(performance, on='emp_id', how='left')
print("Full merged:", full.shape)
print(full[['name','dept_name','salary','q1_score','q2_score']].head(5))

## 3. Concat — Stack Rows

In [ ]:
new_hires = pd.DataFrame({
    'emp_id':[1021,1022,1023],'name':['Emp_20','Emp_21','Emp_22'],
    'dept_id':[10,30,20],'salary':[55000,72000,48000],'hire_year':[2024,2024,2024],
})
all_emp = pd.concat([employees, new_hires], ignore_index=True)
print(f"After concat: {len(employees)} + {len(new_hires)} = {len(all_emp)} rows")

## 4. melt — Wide to Long

In [ ]:
print("WIDE format:\n", performance.head(3))
perf_long = performance.melt(id_vars='emp_id',
    value_vars=['q1_score','q2_score','q3_score','q4_score'],
    var_name='quarter', value_name='score')
perf_long['quarter'] = perf_long['quarter'].str.replace('_score','').str.upper()
print("\nLONG format:\n", perf_long.head(8))

## 5. pivot_table

In [ ]:
pt = pd.pivot_table(full, values='salary', index='dept_name',
                   columns='hire_year', aggfunc='mean', fill_value=0).round(0)
print("Avg Salary — Dept × Hire Year:\n", pt)

## 6. groupby + transform

In [ ]:
full['dept_avg']     = full.groupby('dept_name')['salary'].transform('mean').round(0)
full['vs_avg']       = (full['salary'] - full['dept_avg']).round(0)
full['dept_rank']    = full.groupby('dept_name')['salary'].rank(ascending=False).astype(int)
print(full[['name','dept_name','salary','dept_avg','vs_avg','dept_rank']].head(8))

## 7. Binning — cut() and qcut()

In [ ]:
full['band']     = pd.cut(full['salary'],    bins=[0,50000,70000,90000,200000],
                            labels=['Low','Medium','High','Very High'])
full['quartile'] = pd.qcut(full['salary'],  q=4,
                            labels=['Q1','Q2','Q3','Q4'])
print("Band distribution (cut):")
print(full['band'].value_counts().sort_index())
print("\nQuartile distribution (qcut):")
print(full['quartile'].value_counts().sort_index())

## 8. String Operations

In [ ]:
df_str = pd.DataFrame({
    'raw_name':['  alice KUMAR  ','BOB Singh','charlie NAIR '],
    'email':   ['alice@gmail.com','bob@yahoo.com','charlie@gmail.com'],
    'phone':   ['9876543210','  9123456780','98-76-54-3211'],
    'skills':  ['Python,SQL,Excel','Java,Python,AWS','SQL,Tableau,R'],
})
df_str['name_clean']   = df_str['raw_name'].str.strip().str.title()
df_str['email_domain'] = df_str['email'].str.split('@').str[1]
df_str['phone_clean']  = df_str['phone'].str.strip().str.replace(r'[^0-9]','',regex=True)
df_str['skill_count']  = df_str['skills'].str.split(',').str.len()
df_str['has_python']   = df_str['skills'].str.contains('Python')
print(df_str[['name_clean','email_domain','phone_clean','skill_count','has_python']])

## 9. Visualize

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 5))
fig.suptitle('After Wrangling', fontsize=13, fontweight='bold')
band_counts = full['band'].value_counts().sort_index()
axes[0].bar(band_counts.index.astype(str), band_counts.values, color='steelblue', edgecolor='white')
axes[0].set_title('Salary Band Distribution')
perf_avg = perf_long.groupby('quarter')['score'].mean()
axes[1].bar(perf_avg.index, perf_avg.values, color='coral', edgecolor='white')
axes[1].set_title('Avg Score by Quarter')
plt.tight_layout()
plt.savefig('/home/claude/data_analysis_portfolio/notebooks/06_wrangling.png', dpi=100)
plt.show()

---
## ✅ Summary
| Technique | Method |
|-----------|--------|
| Join tables | `merge()` |
| Stack rows | `pd.concat()` |
| Wide→Long | `melt()` |
| Long→Wide | `pivot_table()` |
| Group stats | `groupby().transform()` |
| Fixed bins | `pd.cut()` |
| Equal freq | `pd.qcut()` |
| String ops | `.str.strip()` `.str.contains()` |